In [1]:
import pandas as pd
import numpy as np

import os

In [2]:
# setup credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = (
    os.path.abspath("../credentials/github.txt"))

%load_ext google.cloud.bigquery

### Notes

[Reference](https://arxiv.org/pdf/1911.00169.pdf)

- 위 논문에서는 데이터를 block, trace, receipt로 구분
- bigquery에서는 어떻게 매칭되는지 잘 모르겟음
- trace는 그 trace가 맞는 거 같음

**Trace**
- Trace data cannot be directly obtained or observed from the contract execution
  - only appears during the execution
- Trace Types
  - Create: when a smart contract is deployed
  - Call: money or message are transferred through different Ethereum address
  - Suicide: smart contract deletes its code
  - Reward: miners get the Ether

**Trace index**
```
A
  CALLs B
    CALLs G
  CALLs C
    CALLs G
```
then it should look something like:
```
[ {A: []}, {B: [0]}, {G: [0, 0]}, {C: [1]}, {G: [1, 0]} ]
```

[Trace reference](https://medium.com/google-cloud/how-to-query-balances-for-all-ethereum-addresses-in-bigquery-fb594e4034a7)

**Internal Transaction**
- If someone asks a smart. contract t osend 10 Ehters to another one, 
  - the Ether transaction from the contract will not be observed in the block -> Internal transaction
  
  

### Transferring Ethereum

In [62]:
%%bigquery trace_df
select *
from `bigquery-public-data.crypto_ethereum.traces`
where DATE(block_timestamp) = "2020-06-07"
and transaction_hash = '0xc7fd515ec6c10802baccc8de6e93fb24baf5d3c0e033728b161293e842ace70e'

Downloading: 100%|██████████| 1/1 [00:03<00:00,  3.64s/rows]


In [63]:
trace_df

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
0,0xc7fd515ec6c10802baccc8de6e93fb24baf5d3c0e033...,86,0x97122ddca38c29b7653d52b07998d06a7128fa0b,0xb34a4271906780955f437bb982f1f39685ac7940,875100000000000000.000000000,0x,0x,call,call,None,79000,0,0,None,None,1,2020-06-07 09:42:18+00:00,10217910,0x5676b7a68ea460696352712d90fbc847e72dda3f387f...,call_0xc7fd515ec6c10802baccc8de6e93fb24baf5d3c...


In [64]:
trace_df.iloc[0]

transaction_hash     0xc7fd515ec6c10802baccc8de6e93fb24baf5d3c0e033...
transaction_index                                                   86
from_address                0x97122ddca38c29b7653d52b07998d06a7128fa0b
to_address                  0xb34a4271906780955f437bb982f1f39685ac7940
value                                     875100000000000000.000000000
input                                                               0x
output                                                              0x
trace_type                                                        call
call_type                                                         call
reward_type                                                       None
gas                                                              79000
gas_used                                                             0
subtraces                                                            0
trace_address                                                     None
error 

In [65]:
%%bigquery transfer_df
select *
from `bigquery-public-data.crypto_ethereum.token_transfers`
where DATE(block_timestamp) = "2020-06-07"
and transaction_hash = '0xc7fd515ec6c10802baccc8de6e93fb24baf5d3c0e033728b161293e842ace70e'

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 410.52query/s]                          
Downloading: 0rows [00:01, ?rows/s]


In [74]:
transfer_df

,token_address,from_address,to_address,value,transaction_hash,log_index,block_timestamp,block_number,block_hash


In [70]:
%%bigquery transaction_df
select *
from `bigquery-public-data.crypto_ethereum.transactions`
where DATE(block_timestamp) = "2020-06-07"
and (
    (from_address = '0xb34a4271906780955f437bb982f1f39685ac7940') or
    (to_address = '0xb34a4271906780955f437bb982f1f39685ac7940')
    )

Downloading: 100%|██████████| 1/1 [00:03<00:00,  3.51s/rows]


In [71]:
transaction_df

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash
0,0xc7fd515ec6c10802baccc8de6e93fb24baf5d3c0e033...,114541,86,0x97122ddca38c29b7653d52b07998d06a7128fa0b,0xb34a4271906780955f437bb982f1f39685ac7940,875100000000000000.000000000,100000,51000000000,0x,3438927,21000,None,None,1,2020-06-07 09:42:18+00:00,10217910,0x5676b7a68ea460696352712d90fbc847e72dda3f387f...


In [73]:
transaction_df.iloc[0]

hash                           0xc7fd515ec6c10802baccc8de6e93fb24baf5d3c0e033...
nonce                                                                     114541
transaction_index                                                             86
from_address                          0x97122ddca38c29b7653d52b07998d06a7128fa0b
to_address                            0xb34a4271906780955f437bb982f1f39685ac7940
value                                               875100000000000000.000000000
gas                                                                       100000
gas_price                                                            51000000000
input                                                                         0x
receipt_cumulative_gas_used                                              3438927
receipt_gas_used                                                           21000
receipt_contract_address                                                    None
receipt_root                

- 왜 transfer가 없지? ethereum은 기록 안되는 것?

### Buying Axs

In [76]:
%%bigquery trace_df
select *
from `bigquery-public-data.crypto_ethereum.traces`
where DATE(block_timestamp) = "2020-06-14"
and transaction_hash in (
    '0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c06f39b76d66d93f0f81267eb8',
    '0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b023a77a44f66d6f067a3d9488',
    '0x57116238d0e7da9ecdfb1ab1c95525e20c525f3a0df788909a34e6973800422d'
)

Downloading: 100%|██████████| 37/37 [00:03<00:00,  9.30rows/s]


In [83]:
address_dic = {
    '0x7be8076f4ea4a4ad08075c2508e481d6c946d12b':'OpenSea', 
    '0x5b3256965e7C3cF26E11FCAf296DfC8807C01073':'OpenSea: Wallet'
    '0xb34a4271906780955f437bb982f1f39685ac7940':'Nuree_metamask',
    '0xa5409ec958c83c3f309868babaca7c86dcb077c1':'Axs contract'
}
trace_df.from_address = trace_df.from_address.replace(address_dic)
trace_df.to_address = trace_df.to_address.replace(address_dic)

In [84]:
for t_hash, group in trace_df.groupby('transaction_hash'):
    display(group)

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
0,0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c06f39...,123,opensea,Axs,0E-9,0x97204d8e,0x000000000000000000000000f9e266af4bca5890e278...,call,call,None,189687,1313,0,1,None,1,2020-06-14 15:51:04+00:00,10264786,0xb719c3229267628edce12c6081f1de94229c4c15ba68...,call_0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c...
4,0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c06f39...,123,opensea,0x7762a5d2a8478bbcc348cd039d93845ea1b142d3,0E-9,0x5c60da1b,0x000000000000000000000000f9e266af4bca5890e278...,call,call,None,186749,1225,0,2,None,1,2020-06-14 15:51:04+00:00,10264786,0xb719c3229267628edce12c6081f1de94229c4c15ba68...,call_0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c...
5,0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c06f39...,123,opensea,0x0d4ddea3849c54e16c85cb4354943dc876e05552,28725000000000000.000000000,0x,0x,call,call,None,2300,0,0,4,None,1,2020-06-14 15:51:04+00:00,10264786,0xb719c3229267628edce12c6081f1de94229c4c15ba68...,call_0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c...
6,0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c06f39...,123,0x7762a5d2a8478bbcc348cd039d93845ea1b142d3,0xf5b0a3efb8e8e4c201e2a935f110eaaf3ffecb8d,0E-9,0x23b872dd0000000000000000000000000d4ddea3849c...,0x,call,call,None,131314,106501,1,"5,0,1",None,1,2020-06-14 15:51:04+00:00,10264786,0xb719c3229267628edce12c6081f1de94229c4c15ba68...,call_0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c...
7,0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c06f39...,123,0x7762a5d2a8478bbcc348cd039d93845ea1b142d3,0xf9e266af4bca5890e2781812cc6a6e89495a79f2,0E-9,0x1b0f7ba9000000000000000000000000f5b0a3efb8e8...,0x00000000000000000000000000000000000000000000...,call,delegatecall,None,140802,113998,2,"5,0",None,1,2020-06-14 15:51:04+00:00,10264786,0xb719c3229267628edce12c6081f1de94229c4c15ba68...,call_0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c...
8,0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c06f39...,123,Nuree_metamask,opensea,30000000000000000.000000000,0xab834bab0000000000000000000000007be8076f4ea4...,0x,call,call,None,236019,209776,6,None,None,1,2020-06-14 15:51:04+00:00,10264786,0xb719c3229267628edce12c6081f1de94229c4c15ba68...,call_0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c...
9,0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c06f39...,123,opensea,Axs,0E-9,0xc45527910000000000000000000000000d4ddea3849c...,0x0000000000000000000000007762a5d2a8478bbcc348...,call,call,None,193817,1482,0,0,None,1,2020-06-14 15:51:04+00:00,10264786,0xb719c3229267628edce12c6081f1de94229c4c15ba68...,call_0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c...
11,0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c06f39...,123,0xe8bd438d0383cf4d19641eaa4793eddc6cebeaf1,0xf5b0a3efb8e8e4c201e2a935f110eaaf3ffecb8d,0E-9,0x5c975abb,0x00000000000000000000000000000000000000000000...,call,call,None,115552,1628,0,"5,0,1,0,0",None,1,2020-06-14 15:51:04+00:00,10264786,0xb719c3229267628edce12c6081f1de94229c4c15ba68...,call_0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c...
12,0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c06f39...,123,0x7762a5d2a8478bbcc348cd039d93845ea1b142d3,Axs,0E-9,0x69dc9ff30000000000000000000000007be8076f4ea4...,0x00000000000000000000000000000000000000000000...,call,call,None,133996,1253,0,"5,0,0",None,1,2020-06-14 15:51:04+00:00,10264786,0xb719c3229267628edce12c6081f1de94229c4c15ba68...,call_0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c...
16,0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c06f39...,123,opensea,0x7762a5d2a8478bbcc348cd039d93845ea1b142d3,0E-9,0x1b0f7ba9000000000000000000000000f5b0a3efb8e8...,0x00000000000000000000000000000000000000000000...,call,call,None,145051,116058,1,5,None,1,2020-06-14 15:51:04+00:00,10264786,0xb719c3229267628edce12c6081f1de94229c4c15ba68...,call_0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c...


,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
1,0x57116238d0e7da9ecdfb1ab1c95525e20c525f3a0df7...,117,opensea,0x7e04bb871bef53e54b46b40e13ceb3a5c54f7b8e,15033855534357201.000000000,0x,0x,call,call,None,2300,0,0,4,None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0x57116238d0e7da9ecdfb1ab1c95525e20c525f3...
3,0x57116238d0e7da9ecdfb1ab1c95525e20c525f3a0df7...,117,Nuree_metamask,opensea,15701198661143478.000000000,0xab834bab0000000000000000000000007be8076f4ea4...,0x,call,call,None,244948,217884,7,None,None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0x57116238d0e7da9ecdfb1ab1c95525e20c525f3...
10,0x57116238d0e7da9ecdfb1ab1c95525e20c525f3a0df7...,117,opensea,0x87768b7016c8b1dfdd029bc69acbdec43b924f30,0E-9,0x1b0f7ba9000000000000000000000000f5b0a3efb8e8...,0x00000000000000000000000000000000000000000000...,call,call,None,145859,116058,1,6,None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0x57116238d0e7da9ecdfb1ab1c95525e20c525f3...
14,0x57116238d0e7da9ecdfb1ab1c95525e20c525f3a0df7...,117,opensea,Axs,0E-9,0xc45527910000000000000000000000007e04bb871bef...,0x00000000000000000000000087768b7016c8b1dfdd02...,call,call,None,202598,1482,0,0,None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0x57116238d0e7da9ecdfb1ab1c95525e20c525f3...
17,0x57116238d0e7da9ecdfb1ab1c95525e20c525f3a0df7...,117,0x87768b7016c8b1dfdd029bc69acbdec43b924f30,0xf9e266af4bca5890e2781812cc6a6e89495a79f2,0E-9,0x1b0f7ba9000000000000000000000000f5b0a3efb8e8...,0x00000000000000000000000000000000000000000000...,call,delegatecall,None,141597,113998,2,"6,0",None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0x57116238d0e7da9ecdfb1ab1c95525e20c525f3...
18,0x57116238d0e7da9ecdfb1ab1c95525e20c525f3a0df7...,117,0x87768b7016c8b1dfdd029bc69acbdec43b924f30,Axs,0E-9,0x69dc9ff30000000000000000000000007be8076f4ea4...,0x00000000000000000000000000000000000000000000...,call,call,None,134778,1253,0,"6,0,0",None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0x57116238d0e7da9ecdfb1ab1c95525e20c525f3...
21,0x57116238d0e7da9ecdfb1ab1c95525e20c525f3a0df7...,117,0x87768b7016c8b1dfdd029bc69acbdec43b924f30,0xf5b0a3efb8e8e4c201e2a935f110eaaf3ffecb8d,0E-9,0x23b872dd0000000000000000000000007e04bb871bef...,0x,call,call,None,132097,106501,1,"6,0,1",None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0x57116238d0e7da9ecdfb1ab1c95525e20c525f3...
22,0x57116238d0e7da9ecdfb1ab1c95525e20c525f3a0df7...,117,0xf5b0a3efb8e8e4c201e2a935f110eaaf3ffecb8d,0xe8bd438d0383cf4d19641eaa4793eddc6cebeaf1,0E-9,0xf7ebc39a0000000000000000000000007e04bb871bef...,0x00000000000000000000000000000000000000000000...,call,call,None,121126,4758,1,"6,0,1,0",None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0x57116238d0e7da9ecdfb1ab1c95525e20c525f3...
24,0x57116238d0e7da9ecdfb1ab1c95525e20c525f3a0df7...,117,opensea,0x5b3256965e7c3cf26e11fcaf296dfc8807c01073,667299070715593.000000000,0x,0x,call,call,None,2300,0,0,3,None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0x57116238d0e7da9ecdfb1ab1c95525e20c525f3...
32,0x57116238d0e7da9ecdfb1ab1c95525e20c525f3a0df7...,117,opensea,Axs,0E-9,0x97204d8e,0x000000000000000000000000f9e266af4bca5890e278...,call,call,None,198467,1313,0,1,None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0x57116238d0e7da9ecdfb1ab1c95525e20c525f3...


,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
2,0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b023a7...,22,opensea,0x7762a5d2a8478bbcc348cd039d93845ea1b142d3,0E-9,0x1b0f7ba9000000000000000000000000f5b0a3efb8e8...,0x00000000000000000000000000000000000000000000...,call,call,None,145051,116058,1,5,None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b...
13,0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b023a7...,22,Nuree_metamask,opensea,30000000000000000.000000000,0xab834bab0000000000000000000000007be8076f4ea4...,0x,call,call,None,236019,209776,6,None,None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b...
15,0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b023a7...,22,opensea,0x7762a5d2a8478bbcc348cd039d93845ea1b142d3,0E-9,0x5c60da1b,0x000000000000000000000000f9e266af4bca5890e278...,call,call,None,186749,1225,0,2,None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b...
20,0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b023a7...,22,0x7762a5d2a8478bbcc348cd039d93845ea1b142d3,0xf5b0a3efb8e8e4c201e2a935f110eaaf3ffecb8d,0E-9,0x23b872dd0000000000000000000000000d4ddea3849c...,0x,call,call,None,131314,106501,1,"5,0,1",None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b...
23,0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b023a7...,22,opensea,0x5b3256965e7c3cf26e11fcaf296dfc8807c01073,1275000000000000.000000000,0x,0x,call,call,None,2300,0,0,3,None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b...
25,0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b023a7...,22,opensea,0x0d4ddea3849c54e16c85cb4354943dc876e05552,28725000000000000.000000000,0x,0x,call,call,None,2300,0,0,4,None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b...
26,0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b023a7...,22,0x7762a5d2a8478bbcc348cd039d93845ea1b142d3,0xf9e266af4bca5890e2781812cc6a6e89495a79f2,0E-9,0x1b0f7ba9000000000000000000000000f5b0a3efb8e8...,0x00000000000000000000000000000000000000000000...,call,delegatecall,None,140802,113998,2,"5,0",None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b...
27,0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b023a7...,22,0xe8bd438d0383cf4d19641eaa4793eddc6cebeaf1,0xf5b0a3efb8e8e4c201e2a935f110eaaf3ffecb8d,0E-9,0x5c975abb,0x00000000000000000000000000000000000000000000...,call,call,None,115552,1628,0,"5,0,1,0,0",None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b...
28,0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b023a7...,22,0xf5b0a3efb8e8e4c201e2a935f110eaaf3ffecb8d,0xe8bd438d0383cf4d19641eaa4793eddc6cebeaf1,0E-9,0xf7ebc39a0000000000000000000000000d4ddea3849c...,0x00000000000000000000000000000000000000000000...,call,call,None,120355,4758,1,"5,0,1,0",None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b...
29,0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b023a7...,22,opensea,Axs,0E-9,0x97204d8e,0x000000000000000000000000f9e266af4bca5890e278...,call,call,None,189687,1313,0,1,None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...,call_0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b...


In [85]:
%%bigquery transfer_df
select *
from `bigquery-public-data.crypto_ethereum.token_transfers`
where DATE(block_timestamp) = "2020-06-14"
and transaction_hash in (
    '0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c06f39b76d66d93f0f81267eb8',
    '0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b023a77a44f66d6f067a3d9488',
    '0x57116238d0e7da9ecdfb1ab1c95525e20c525f3a0df788909a34e6973800422d'
)

Downloading: 100%|██████████| 3/3 [00:03<00:00,  1.17s/rows]


In [86]:
transfer_df

,token_address,from_address,to_address,value,transaction_hash,log_index,block_timestamp,block_number,block_hash
0,0xf5b0a3efb8e8e4c201e2a935f110eaaf3ffecb8d,0x0d4ddea3849c54e16c85cb4354943dc876e05552,0xb34a4271906780955f437bb982f1f39685ac7940,46432,0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b023a7...,16,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...
1,0xf5b0a3efb8e8e4c201e2a935f110eaaf3ffecb8d,0x7e04bb871bef53e54b46b40e13ceb3a5c54f7b8e,0xb34a4271906780955f437bb982f1f39685ac7940,143766,0x57116238d0e7da9ecdfb1ab1c95525e20c525f3a0df7...,96,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...
2,0xf5b0a3efb8e8e4c201e2a935f110eaaf3ffecb8d,0x0d4ddea3849c54e16c85cb4354943dc876e05552,0xb34a4271906780955f437bb982f1f39685ac7940,50221,0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c06f39...,101,2020-06-14 15:51:04+00:00,10264786,0xb719c3229267628edce12c6081f1de94229c4c15ba68...


In [89]:
%%bigquery transaction_df
select *
from `bigquery-public-data.crypto_ethereum.transactions`
where DATE(block_timestamp) = "2020-06-14"
and `hash` in (
    '0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c06f39b76d66d93f0f81267eb8',
    '0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b023a77a44f66d6f067a3d9488',
    '0x57116238d0e7da9ecdfb1ab1c95525e20c525f3a0df788909a34e6973800422d'
)

Downloading: 100%|██████████| 3/3 [00:03<00:00,  1.16s/rows]


In [90]:
transaction_df

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash
0,0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b023a7...,1,22,0xb34a4271906780955f437bb982f1f39685ac7940,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,30000000000000000.000000000,272815,27000000000,0xab834bab0000000000000000000000007be8076f4ea4...,1562794,212372,None,None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...
1,0x57116238d0e7da9ecdfb1ab1c95525e20c525f3a0df7...,2,117,0xb34a4271906780955f437bb982f1f39685ac7940,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,15701198661143478.000000000,281852,26000000000,0xab834bab0000000000000000000000007be8076f4ea4...,6938327,220588,None,None,1,2020-06-14 16:11:29+00:00,10264880,0x21fcd95b7386e21ae8f2ae82e896afc4922d2c52b9d0...
2,0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c06f39...,0,123,0xb34a4271906780955f437bb982f1f39685ac7940,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,30000000000000000.000000000,272815,26000000000,0xab834bab0000000000000000000000007be8076f4ea4...,6276642,212372,None,None,1,2020-06-14 15:51:04+00:00,10264786,0xb719c3229267628edce12c6081f1de94229c4c15ba68...


In [91]:
trace_df.groupby('transaction_hash').gas.sum()

transaction_hash
0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c06f39b76d66d93f0f81267eb8    1597942
0x57116238d0e7da9ecdfb1ab1c95525e20c525f3a0df788909a34e6973800422d    1640211
0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b023a77a44f66d6f067a3d9488    1597942
Name: gas, dtype: int64

In [95]:
trace_df.groupby('transaction_hash').gas_used.sum()

transaction_hash
0x37443e2cb01a1ef74ac1e1ed4ac78cdb3ab718c06f39b76d66d93f0f81267eb8    557992
0x57116238d0e7da9ecdfb1ab1c95525e20c525f3a0df788909a34e6973800422d    566100
0xc6a5a87e0b7f1e72129f75e899f79fee7100b6b023a77a44f66d6f067a3d9488    557992
Name: gas_used, dtype: int64

- 왜케 trace가 복잡하지
- 가스는 어떻게 계산된 거지
- transfer에는 ethereum말고 다른 것들이 기록되는 건가?
- anomic match ... 는 뭐지 뒤에 짤린 건 뭘까

### 아바타 구매

In [96]:
%%bigquery trace_df
select *
from `bigquery-public-data.crypto_ethereum.traces`
where DATE(block_timestamp) = "2020-06-21"
and transaction_hash = '0x4212c1d67a5d297bf25ee7100234f5056fc25777db427b466e2f76de6cc86314'

Downloading: 100%|██████████| 1/1 [00:03<00:00,  3.70s/rows]


In [97]:
trace_df

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
0,0x4212c1d67a5d297bf25ee7100234f5056fc25777db42...,103,0xb34a4271906780955f437bb982f1f39685ac7940,0xe31763aad9294f073ddf18b36503ed037ae5e737,420000000000000000.000000000,0xd0e30db0,0x,call,call,None,55970,30292,0,None,None,1,2020-06-21 05:29:11+00:00,10307239,0x998dfa328349819ad3c124ba9a775497b05753ff7869...,call_0x4212c1d67a5d297bf25ee7100234f5056fc2577...


In [98]:
%%bigquery transfer_df
select *
from `bigquery-public-data.crypto_ethereum.token_transfers`
where DATE(block_timestamp) = "2020-06-21"
and transaction_hash = '0x4212c1d67a5d297bf25ee7100234f5056fc25777db427b466e2f76de6cc86314'

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 271.88query/s]                          
Downloading: 0rows [00:02, ?rows/s]


In [99]:
transfer_df

,token_address,from_address,to_address,value,transaction_hash,log_index,block_timestamp,block_number,block_hash


In [101]:
%%bigquery transaction_df
select *
from `bigquery-public-data.crypto_ethereum.transactions`
where DATE(block_timestamp) = "2020-06-21"
and `hash` = '0x4212c1d67a5d297bf25ee7100234f5056fc25777db427b466e2f76de6cc86314'

Downloading: 100%|██████████| 1/1 [00:03<00:00,  3.61s/rows]


In [102]:
transaction_df

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash
0,0x4212c1d67a5d297bf25ee7100234f5056fc25777db42...,3,103,0xb34a4271906780955f437bb982f1f39685ac7940,0xe31763aad9294f073ddf18b36503ed037ae5e737,420000000000000000.000000000,77034,19000000000,0xd0e30db0,4915824,51356,None,None,1,2020-06-21 05:29:11+00:00,10307239,0x998dfa328349819ad3c124ba9a775497b05753ff7869...


- etherscan은 어떻게 deposit이라고 해석했지?
- 왜 token transfer는 없지?
- transaction fee는 여기서 어떻게 파싱하지?
- 7329와 7330은 왜 기록 안됐지
- 일반적으로 이렇게 구매한 스마트 컨트랙트를 어떻게 구분하지  
  -> ML: node clustering?  
  -> data parsing?
  
  

### 아바타 매물로 내놓기 (Registry)

In [105]:
%%bigquery trace_df
select *
from `bigquery-public-data.crypto_ethereum.traces`
where DATE(block_timestamp) = "2020-06-21"
and transaction_hash = '0xc0d7f363c2b42e0044dc7568003f5102993ffa71ec4aeb15229965a391e991a9'

Downloading: 100%|██████████| 3/3 [00:02<00:00,  1.02rows/s]


In [110]:
trace_df

,transaction_hash,transaction_index,from_address,to_address,value,input,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,block_timestamp,block_number,block_hash,trace_id
0,0xc0d7f363c2b42e0044dc7568003f5102993ffa71ec4a...,91,0xa5409ec958c83c3f309868babaca7c86dcb077c1,0x506ae7f09042f4bd73f2494d6b1feec364b80efe,0E-9,0x608060405234801561001057600080fd5b5060405161...,0x6080604052600436106100825763ffffffff7c010000...,create,None,None,368036,313583,1,0,None,1,2020-06-21 14:21:31+00:00,10309617,0x6b718e3c47100de2d2b44a567674ba950e14a75d0d19...,create_0xc0d7f363c2b42e0044dc7568003f5102993ff...
1,0xc0d7f363c2b42e0044dc7568003f5102993ffa71ec4a...,91,0xb34a4271906780955f437bb982f1f39685ac7940,0xa5409ec958c83c3f309868babaca7c86dcb077c1,0E-9,0xddd81f82,0x000000000000000000000000506ae7f09042f4bd73f2...,call,call,None,409479,370338,1,None,None,1,2020-06-21 14:21:31+00:00,10309617,0x6b718e3c47100de2d2b44a567674ba950e14a75d0d19...,call_0xc0d7f363c2b42e0044dc7568003f5102993ffa7...
2,0xc0d7f363c2b42e0044dc7568003f5102993ffa71ec4a...,91,0x506ae7f09042f4bd73f2494d6b1feec364b80efe,0xf9e266af4bca5890e2781812cc6a6e89495a79f2,0E-9,0x485cc955000000000000000000000000b34a42719067...,0x,call,delegatecall,None,317467,44620,0,"0,0",None,1,2020-06-21 14:21:31+00:00,10309617,0x6b718e3c47100de2d2b44a567674ba950e14a75d0d19...,call_0xc0d7f363c2b42e0044dc7568003f5102993ffa7...


In [109]:
%%bigquery transfer_df
select *
from `bigquery-public-data.crypto_ethereum.token_transfers`
where DATE(block_timestamp) = "2020-06-21"
and transaction_hash = '0xc0d7f363c2b42e0044dc7568003f5102993ffa71ec4aeb15229965a391e991a9'

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 373.56query/s]                          
Downloading: 0rows [00:01, ?rows/s]


In [111]:
transfer_df

,token_address,from_address,to_address,value,transaction_hash,log_index,block_timestamp,block_number,block_hash


In [112]:
%%bigquery transaction_df
select *
from `bigquery-public-data.crypto_ethereum.transactions`
where DATE(block_timestamp) = "2020-06-21"
and `hash` = '0xc0d7f363c2b42e0044dc7568003f5102993ffa71ec4aeb15229965a391e991a9'

Downloading: 100%|██████████| 1/1 [00:03<00:00,  3.61s/rows]


In [113]:
transaction_df

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash
0,0xc0d7f363c2b42e0044dc7568003f5102993ffa71ec4a...,4,91,0xb34a4271906780955f437bb982f1f39685ac7940,0xa5409ec958c83c3f309868babaca7c86dcb077c1,0E-9,430543,36000000000,0xddd81f82,4222204,391402,None,None,1,2020-06-21 14:21:31+00:00,10309617,0x6b718e3c47100de2d2b44a567674ba950e14a75d0d19...


In [ ]:
0xfae0a1e620342d83b200389adb0689eac04eb2d0f4d78aaeb6d15905375e4b35

## Next to do

- Balance table
  - daily로 각 address의 잔고 보여줌
  - 잔고 많은 주요 유저들 탐색 가능